 - Life Lone Learning
 - Elastic Weight Consolidation (EWC)
    - [LINK](https://www.kaggle.com/code/hughiephan/elastic-weight-consolidation-on-minst)
    - 公式 $\mathcal{L}(\theta) = \mathcal{L}_B(\theta) + \frac\lambda2\sum^n_{i=1}F_i(\theta_i - \theta^*_{A,i})^2$
        - B是指目前任務，A是指前一個任務

In [ ]:
import torch
from torch import nn